In [ ]:
!pip install -e .
from adaptive_ensemble.data import load_and_prepare
from adaptive_ensemble.context import ContextDetectorSK
from adaptive_ensemble.learners import BaseLearners
from adaptive_ensemble.agent import QLearningAgent
from adaptive_ensemble.ensemble import add_context

In [ ]:
# ==============================
# گام 3: بارگذاری و آماده‌سازی داده از Google Drive
# ==============================
from google.colab import drive
drive.mount('/content/drive')

csv_path = '/content/drive/MyDrive/yelp.csv'

from adaptive_ensemble.data import load_and_prepare

train_df, test_df = load_and_prepare(csv_path, test_size=0.2, random_state=42)

print(f"Train set: {train_df.shape}")
print(f"Test set : {test_df.shape}")

train_df.head()


In [ ]:
# ساخت detector با تعداد تاپیک k_topics=10
detector = ContextDetectorSK(k_topics=10, max_features=5000)

# آموزش روی متن‌های train
detector.fit(train_df.clean_text.tolist())

# تست کوچکی
ctx_vec = detector(train_df.clean_text.iloc[0], train_df.date.iloc[0], train_df.user_id.iloc[0])
print("Context vector shape:", ctx_vec.shape)

In [ ]:
# اضافه کردن ستون‌های context
train_ctx = add_context(train_df, detector)
test_ctx  = add_context(test_df, detector)

# بررسی شکل داده
print(train_ctx.columns[-5:], train_ctx.shape)

In [ ]:
import torch

# تشخیص device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Base learners
bl = BaseLearners(device)
bl.fit_svm(train_df.clean_text.tolist(), train_df.label.tolist())

# پارامترهای state و action
state_dim  = train_ctx.filter(like="context_").shape[1]
action_dim = len(bl.predict_all(["dummy"])) + 1  # blend action

agent = QLearningAgent(state_dim, action_dim)

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import time
import matplotlib.pyplot as plt

# پاره‌ای پارامترها
num_epochs, patience = 10, 3
batch_size, target_update = 64, 50

# آماده‌سازی داده‌ها
X = train_ctx.filter(like="context_").values
y = train_ctx.label.values
val_n = int(0.1 * len(X))
X_val, y_val = X[:val_n], y[:val_n]
X_tr, y_tr = X[val_n:], y[val_n:]

rewards, best_f1, no_imp, step = [], 0, 0, 0
start = time.time()

for ep in range(1, num_epochs+1):
    total_r = 0
    for i, s in enumerate(X_tr):
        models = bl.predict_all([train_df.clean_text.iloc[val_n+i]])
        action = agent.select_action(s)
        # ... (reward shaping as before) ...
        # فرض کنید reward محاسبه می‌شود
        reward = 1.0  # نمونه
        total_r += reward
        agent.store(s, action, reward, s)
        agent.optimize(batch_size)
        if step % target_update == 0:
            agent.update_target()
        agent.eps *= 0.99
        step += 1
    rewards.append(total_r)

    # اعتبارسنجی ساده
    y_pred = [agent.select_action(s) for s in X_val]
    _,_,f1,_ = precision_recall_fscore_support(y_val, y_pred, average='macro')
    print(f"Epoch {ep}: Reward={total_r:.1f}, F1={f1:.4f}")

    if f1 > best_f1:
        best_f1, no_imp = f1, 0
    else:
        no_imp += 1
        if no_imp >= patience:
            break

print("Training done in", round((time.time()-start)/60,2), "min")

# نمایش نمودار پاداش
plt.plot(rewards)
plt.title("Rewards per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Total Reward")
plt.show()